In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, Dot, Add, Dense, Dropout, Concatenate
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sample import get_saved_samples

In [2]:
# Assuming 'getSavedSamples()' is defined elsewhere and available
users, ratings, books = get_saved_samples()

In [3]:
nUsers = ratings['User-ID'].nunique()
nBooks = ratings['ISBN'].nunique()

# Embedding size and regularization parameter
embeddingSize = 15
regularization = 1e-6

In [4]:
# User Embeddings
userInput = Input(shape=(1,))
userEmbedding = Embedding(nUsers, embeddingSize, embeddings_regularizer=l2(regularization), input_length=1)(userInput)
userVector = Flatten()(userEmbedding)
userBias = Embedding(nUsers, 1, input_length=1)(userInput)
userBias = Flatten()(userBias)

# Book Embeddings
bookInput = Input(shape=(1,))
bookEmbedding = Embedding(nBooks, embeddingSize, embeddings_regularizer=l2(regularization), input_length=1)(bookInput)
bookVector = Flatten()(bookEmbedding)
bookBias = Embedding(nBooks, 1, input_length=1)(bookInput)
bookBias = Flatten()(bookBias)

In [5]:
# Concatenate user and book vectors
concatenated = Concatenate()([userVector, bookVector])

# Add fully connected layers
dense1 = Dense(64, activation='relu')(concatenated)
dropout1 = Dropout(0.2)(dense1)
dense2 = Dense(32, activation='relu')(dropout1)
dropout2 = Dropout(0.2)(dense2)
output = Dense(1)(dropout2)

# Add biases
output = Add()([output, userBias, bookBias])

# Construct the model
model = Model(inputs=[userInput, bookInput], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

In [6]:
# %%
# Preparing the dataset for training
userIdEncoder = LabelEncoder()
userIds = userIdEncoder.fit_transform(ratings['User-ID'].astype(str))

bookIdEncoder = LabelEncoder()
bookIds = bookIdEncoder.fit_transform(ratings['ISBN'].astype(str))
ratings = ratings['Book-Rating'].values

# Train the model
model.fit([userIds, bookIds], ratings, epochs=10, validation_split=0.2)

Epoch 1/10
2695/2695 [==============================] - 46s 17ms/step - loss: 12.4392 - val_loss: 15.5754
Epoch 2/10
2695/2695 [==============================] - 43s 16ms/step - loss: 8.3046 - val_loss: 18.1543
Epoch 3/10
2695/2695 [==============================] - 44s 16ms/step - loss: 5.7069 - val_loss: 19.5512
Epoch 4/10
2695/2695 [==============================] - 44s 16ms/step - loss: 4.5643 - val_loss: 19.4920
Epoch 5/10
1638/2695 [=================>............] - ETA: 17s - loss: 3.7294